### Table of Content
- [Import Data](#Import-Data)
- [Inspect Data](#Inspect-Data)
- [Clean Data](#Clean-Data)

In [2]:
# import necessary libraries
import pandas as pd
import numpy as np
from pathlib import Path

### Import Data
---------------------------

In this workout, we use a dataset of [cafe sales](https://www.kaggle.com/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training) from kaggle to show how to use data cleaning techniques to process dirty data.

In [3]:
# Load the café sales dataset
data_path = Path("data/dirty_cafe_sales.csv")

# Check if the file exists
if data_path.exists():
    df = pd.read_csv(data_path)
    print(f"Data loaded successfully. {df.shape[0]} rows and {df.shape[1]} columns.")
else:
    raise FileNotFoundError(f"The file {data_path} does not exist.")

Data loaded successfully. 10000 rows and 8 columns.


### Inspect Data
----------------

In [4]:
# Take a first look
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [5]:
print("\nDataFrame Information:")
df.info()

print("\nMissing values in each column:")
df.isnull().sum()


DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB

Missing values in each column:


Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

Through data inspecting we found that:
- column names are not consistent: the names contain white space, lowercase and capital letters.
- the data type of all columns are not correct, because of the messy data values, pandas cannot correctly recognize the data type.
- some columns have wrong values, like `ERROR` and `UNKNOWN`
- some columns have null values. 

So next we will dealing with these problems through data cleaning. 

### Clean Data
--------------

#### Standardize column names

In order to keep the consistency of column names between various tools and database, it's a good practice to remove the whitespace at the beginning and end of the column names, keep all names lowercase and replace the whitespace with underscore `_` in the column names. 

**Objectives**:
- Remove spaces at beginning and end of column names
- Lowercase column names 
- Replace white space between names with `_`


**Methods**: `str.strip()`, `str.lower()`, `str.replace()`.


In [6]:
# --- 1. Standardize column names ---
# Standardize column names: strip whitespace, convert to lowercase, and replace spaces with underscores

df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_', regex=False)
) 

print("\nStandardizing column names:")
df.columns



Standardizing column names:


Index(['transaction_id', 'item', 'quantity', 'price_per_unit', 'total_spent',
       'payment_method', 'location', 'transaction_date'],
      dtype='object')

#### Covert column types

Through the first check of the data sample and the metadata of of the cafe sales dataset, we know that the data type of each column should be like:
| Column           | Data Type category| 
|------------------|-------------------|
| transaction_id   | text              | 
| item             | text              | 
| quantity         | number            | 
| price_per_unit   | number            |
| total_spent      | number            |
| payment_method   | text              |
| location         | text              |
| transaction_date | date              |

So next step we convert the data type of each column.

### Convert numerical columns

**Objectives**:
- Remove commas and spaces in the data values
- Convert data values to numbers 
- Replace invalid values with `NaN`.

**Methods**: `replace()`, `pd.to_numeric()`.

In [7]:
# --- 2. Convert numerical columns ---
num_cols = ['quantity', 'price_per_unit', 'total_spent']
df[num_cols] = (
    df[num_cols]
    .replace(r'[, \s]', '', regex=True)  # Remove commas and spaces
    .apply(pd.to_numeric, errors='coerce') # Convert to numeric, coercing errors to NaN
)

### Convert date column

**Objectives**:
- Convert transaction date strings into datetime objects.
- Convert invalid formats to NaN.

**Methods**: `replace()`, `pd.to_datetime()`.

In [8]:

# --- 3. Convert date columns ---
date_column = 'transaction_date'

# Convert all non-string values to empty string, then replace invalid values with np.nan

df[date_column] = df[date_column].map(lambda x: str(x).strip() if isinstance(x, str) else '')
df[date_column] = df[date_column].replace({'': np.nan, 'unknown': np.nan, 'nan': np.nan}, regex=False)

# Convert to datetime, let pandas infer the format, coerce errors to NaN
df[date_column] = pd.to_datetime(df[date_column], errors='coerce')

### Convert categorical string columns

**Objectives**:
- Remove whitespace 
- Lowercase strings
- Replace invalid strings with `NaN`

**Methods**: `str.strip()`, `str.lower()`, `replace()`.

In [9]:
# --- 4. Convert text columns ---
text_cols = ['transaction_id', 'item', 'payment_method', 'location']
df[text_cols] = df[text_cols].apply(
    lambda x: (x.str.strip()   # Strip whitespace
               .str.lower()  # Convert to lowercase
               .replace({'':np.nan, 'unknown':np.nan, 'nan':np.nan}, regex=False)) # Replace empty strings and 'unknown' with NaN
)

### Check data types after conversion

In [10]:
# Check the data types of the numeric columns
print("\nData types of columns after conversion:")
df.dtypes


Data types of columns after conversion:


transaction_id              object
item                        object
quantity                   float64
price_per_unit             float64
total_spent                float64
payment_method              object
location                    object
transaction_date    datetime64[ns]
dtype: object

In [11]:
df.head()

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
0,txn_1961373,coffee,2.0,2.0,4.0,credit card,takeaway,2023-09-08
1,txn_4977031,cake,4.0,3.0,12.0,cash,in-store,2023-05-16
2,txn_4271903,cookie,4.0,1.0,NaN,credit card,in-store,2023-07-19
3,txn_7034554,salad,2.0,5.0,10.0,NaN,NaN,2023-04-27
4,txn_3160411,coffee,2.0,2.0,4.0,digital wallet,in-store,2023-06-11


### Checking for Duplicates
--------------------------

In [12]:
#--- 5. Check for duplicate rows based on transaction_id ---
print('Duplicate rows:', df.duplicated(subset="transaction_id").sum())

Duplicate rows: 0


### Handling Missing Values
---------------------------

In [ ]:
# --- 6. checking for missing values ---
print("\nMissing values:")
df.isna().sum()


Missing values:


transaction_id         0
item                 677
quantity             479
price_per_unit       533
total_spent          502
payment_method      2872
location            3603
transaction_date     460
dtype: int64

Since there are no extra information for the missing values in column `quantity`, `payment_method`, `location` and `transaction_date`, so the rows with missing values in these columns should be removed. The missing values in `price_per_unit` can be filled according to  the price of other existing items. `total_spent` can be calculated by `quantity * price_per_unit`. The missing `item` can be derived by its unique price. 

In [ ]:
# --- 7. Remove rows with missing values in critical columns ---
drop_cols = ['quantity', 'payment_method', 'location', 'transaction_date']
df = df.dropna(subset=drop_cols)

In [15]:
df.shape

(4186, 8)

In [ ]:
# --- 8. fill missing price_per_unit based on item ---
# create a price mapping dictionary from existing items
price_list = {
    'coffee': 2.0,
    'tea': 1.5,
    'sandwich': 4.0,
    'salad': 5.0,
    'cake': 3.0,
    'cookie': 1.0,
    'smoothie': 4.0,
    'juice': 3.0
}

df['price_per_unit'] = df['price_per_unit'].fillna(df['item'].map(price_list))

In [ ]:
# --- 9. fill missing price_per_unit based on item ---
# create a reverse price mapping dictionary using unique prices
reverse_price_list = {
    2.0:'coffee',
    1.5:'tea',    
    5.0:'salad',   
    1.0:'cookie'    
}
df['item'] = df['item'].fillna(df['price_per_unit'].map(reverse_price_list))

In [18]:
df.sample(10)

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
9136,txn_3587916,salad,3.0,5.0,15.0,cash,in-store,2023-07-09
8781,txn_4267136,tea,1.0,1.5,1.5,credit card,in-store,2023-08-02
303,txn_3723007,tea,3.0,1.5,4.5,digital wallet,in-store,2023-05-18
8566,txn_1245485,cake,1.0,3.0,3.0,digital wallet,error,2023-05-15
6292,txn_6884558,NaN,1.0,3.0,NaN,cash,in-store,2023-08-14
797,txn_5931292,error,2.0,4.0,8.0,credit card,in-store,2023-12-18
7242,txn_9970964,smoothie,2.0,4.0,8.0,credit card,takeaway,2023-07-07
5007,txn_1015883,cookie,4.0,1.0,4.0,cash,in-store,2023-10-16
9584,txn_8955306,coffee,3.0,2.0,6.0,credit card,in-store,2023-10-14
472,txn_3607652,salad,4.0,5.0,20.0,digital wallet,in-store,2023-06-05


In [ ]:
# --- 10. Remove rows with missing values in item and price_per_unit columns ---
drop_cols = ['item', 'price_per_unit']
df = df.dropna(subset=drop_cols)

In [ ]:
# --- 11. recalculate total_spent using cleaned quantity and price_per_unit ---
df['total_spent'] = df['quantity'] * df['price_per_unit']

In [21]:
print("\nMissing values:")
df.isna().sum()


Missing values:


transaction_id      0
item                0
quantity            0
price_per_unit      0
total_spent         0
payment_method      0
location            0
transaction_date    0
dtype: int64

### Final Checks
----------------

In [ ]:
# --- 12. check for negative values in price_per_unit and quantity ---

# check for negative price_per_unit
(df['price_per_unit']<0).sum()

# check for negative quantities
(df['quantity']<0).sum()

0